# Phase 3 - Function Calling
If not already done run this in the top level folder:
```
pip install -r requirements.txt
```




In [ ]:
import json
import requests
import os
import openai
from enum import Enum
from pydantic import BaseModel
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("OPENAI_API_VERSION")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_VERSION")
)
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

This is the object model for receiving questions.

In [ ]:

class QuestionType(str, Enum):
    multiple_choice = "multiple_choice"
    true_false = "true_false"
    popular_choice = "popular_choice"
    estimation = "estimation"

class Ask(BaseModel):
    question: str | None = None
    type: QuestionType
    correlationToken: str | None = None

class Answer(BaseModel):
    answer: str
    correlationToken: str | None = None
    promptTokensUsed: int | None = None
    completionTokensUsed: int | None = None


## Sample Function 

In [ ]:
import pytz
from datetime import datetime

def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."

Make the model aware of the function


In [ ]:
functions = [
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current time in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                        }
                    },
                    "required": ["location"],
                },
            }
        }
    ]   
available_functions = {
            "get_current_time": get_current_time          
        } 

YOUR Mission: Adjust the function below and reuse it in the main.py file later to deploy to Azure and to update your service


In [ ]:

async def ask_question(ask: Ask):
    """
    Ask a question
    """
    
    question = ask.question
    messages= [{"role" : "assistant", "content" : question}
               , { "role" : "system", "content" : "if asked for time answer in the format HH:MM:SS AM/PM only. Do not answer with a full sentence."}
               ]
    first_response = client.chat.completions.create(
        model = deployment_name,
        messages = messages,
        tools = functions,
        tool_choice = "auto",
    )

    print(first_response)
    response_message = first_response.choices[0].message
    tool_calls = response_message.tool_calls

     # Step 2: check if GPT wanted to call a function
    if tool_calls:
        print("Recommended Function call:")
        print(tool_calls)
        print()
    
        # Step 3: call the function
        messages.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            # verify function exists
            if function_name not in available_functions:
                return "Function " + function_name + " does not exist"
            else:
                print("Calling function: " + function_name)
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(function_args)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            ) 
            print("Addding this message to the next prompt:") 
            print (messages)
            
             # extend conversation with function response
            second_response = client.chat.completions.create(
                model = model_name,
                messages = messages)  # get a new response from the model where it can see the function response
            
            print("second_response")
            
            return second_response.choices[0].message.content
            

   

Use this snippet to try your method with several questions.

In [ ]:

ask = Ask(question="What time is it in New York?", type=QuestionType.estimation)
answer = await ask_question(ask)
print('Answer:', answer )


Make sure you transfer your code changes into main.py (or additional files). Then redeploy your container using this command.
```
bash ./azd-hooks/deploy.sh phase1 $AZURE_ENV_NAME
```
Make sure to provide the URL of your endpoint in the team portal!